# Extract string angle from handle image

In [2]:
import numpy as np
import imutils
import cv2
import cv2.aruco as aruco
from numpy.linalg import norm
import math


In [4]:
# Video data
TEST_PATH = '/home/stmoon/Video/CarData3/20181102_163625.mp4'
CAMERA_INFO = './camera.yaml'


In [5]:
def attitude(rvec, tvec ):
    rvec_matrix = cv2.Rodrigues(rvec)[0]
    proj_matrix = np.hstack((rvec_matrix, tvec.T))
    eulerAngles = cv2.decomposeProjectionMatrix(proj_matrix)[6]
    
    #print(eulerAngles)
    yaw   = eulerAngles[1]
    pitch = eulerAngles[0]
    roll  = eulerAngles[2]
#     if roll > 0:
#         roll = 180 - roll  
#     elif roll < 0:
#         roll = -180 - roll 
    yaw = -yaw

    #angle = (angle + 720) % 360

    return roll, pitch, yaw

In [6]:
def handle2steering(handle) :
    return 0.0734*handle - 0.551

In [ ]:
cv_file = cv2.FileStorage(CAMERA_INFO, cv2.FILE_STORAGE_READ)

mtx = cv_file.getNode("camera_matrix").mat()
dist = cv_file.getNode("dist_coeff").mat()

cap = cv2.VideoCapture(TEST_PATH)

# video recorder
fourcc = cv2.VideoWriter_fourcc(*'XVID')
video_writer = cv2.VideoWriter("output.avi", fourcc, 20, (1080, 1920))
    
count = 0

while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()

    
    # Rotate -90 
    frame = np.transpose(frame, axes=(1,0,2)).astype(np.uint8).copy() 
    frame = cv2.flip(frame, 1)
    
    # change image for gray image processing
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    #gray = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 15, 2)

    # aruco information
    aruco_dict = aruco.Dictionary_get(aruco.DICT_4X4_100)
    parameters =  aruco.DetectorParameters_create()

    # lists of ids and the corners beloning to each id
    corners, ids, rejectedImgPoints = aruco.detectMarkers(gray, aruco_dict, parameters=parameters)

    # calculate attitude if marker is found.
    if np.all(ids == 2) :  
        
        rvec, tvec, _ = aruco.estimatePoseSingleMarkers(corners[0], 0.1, mtx, dist)
        
        roll, pitch, yaw = attitude(rvec[0], tvec[0])
#         print(rvec[0])
#         print(roll + 90, pitch, yaw)
        angle = handle2steering(roll+180)

        aruco.drawAxis(frame, mtx, dist, rvec[0], tvec[0], 0.01)
        aruco.drawDetectedMarkers(frame, corners)
    else  :
        continue

    left = 250
    top = 300
    width = 600
    height = 500

    
    # save the image (image_[num]_[angle].png)
    crop_img =  frame[top:top+height, left:left+width]
    fn = "data/out2/image_%04d_%.2f.png" % (count, angle)
    cv2.imwrite( fn, crop_img );

    
    # display image to save 
    cv2.rectangle(frame,(left,top),(left+width,top+height),(0,0,255),3)
    
    
    # display steering angle
    str = "%f"%(roll+180)
    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(frame, str,(700,500), font, 1,(255,255,255),2,cv2.LINE_AA)
    
    # display the resulting frame
    #video_writer.write(frame)
    cv2.imshow('frame', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
       break
    
    count += 1

# when everything done, release the capture
video_writer.release()
cap.release()
cv2.destroyAllWindows()

